In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from pymarket import Market
from pymarket.bids.demand_curves import demand_curve_from_bids, supply_curve_from_bids
from pymarket.plot import plot_demand_curves

# Creates new market

In [2]:
r = np.random.RandomState(1234)
mar = Market()
    
mar.accept_bid(1, 3, 0, True, 0)
mar.accept_bid(2, 4, 1, True, 0)
mar.accept_bid(5, 1, 2, True, 0)

mar.accept_bid(4, 2, 3, False, 0)
mar.accept_bid(1, 1, 4, False, 0)
mar.accept_bid(5, 6, 5, False, 0)
bids = mar.bm.get_df()
transactions, extras = mar.run('muda')

   quantity  price  user  buying  time  divisible
1         2      4     1    True     0       True
4         1      1     4   False     0       True
   quantity  price  user  buying  time  divisible
1         2      4     1    True     0       True <class 'pandas.core.frame.DataFrame'>
   quantity  price  user  buying  time  divisible
0         1      3     0    True     0       True
2         5      1     2    True     0       True
3         4      2     3   False     0       True
5         5      6     5   False     0       True
   quantity  price  user  buying  time  divisible
0         1      3     0    True     0       True
2         5      1     2    True     0       True <class 'pandas.core.frame.DataFrame'>


In [3]:
extras

{'left': [1, 4],
 'right': [0, 2, 3, 5],
 'price_left': 4.0,
 'price_right': 2.0,
 'fees': array([0., 0., 0., 0., 0., 0.])}

In [4]:
bids

,quantity,price,user,buying,time,divisible
0,1,3,0,True,0,True
1,2,4,1,True,0,True
2,5,1,2,True,0,True
3,4,2,3,False,0,True
4,1,1,4,False,0,True
5,5,6,5,False,0,True


In [5]:
transactions.get_df()

,bid,quantity,price,source,active
0,4,1,2.0,-1,False
1,1,1,2.0,-1,False


In [6]:
xx = mar.statistics(reservation_prices={4:0.5})['profits']

In [7]:
xx

{'player_bid': [0, 1.0, 0, 0, 1.0, 0],
 'market': 0.0,
 'player_reservation': [0, 1.0, 0, 0, 1.5, 0]}

In [15]:
y = np.arange(5)

In [16]:
y[xx.index.values]

array([1, 4])

In [23]:
xx.get(4, 0)

1.5

In [9]:
def get_gain(row):
        """Finds the gain of the row

        Parameters
        ----------
        row : TODO

        Returns
        -------
        TODO

        """
        gap = row.price_y - row.price_x
        if not row.buying:
            gap = - gap
        return gap * row.quantity


In [10]:
xx['gain'] = xx.apply(lambda x: get_gain(x), axis=1)

In [14]:
xx.groupby('user')['gain'].sum()

user
1    1.0
4    1.0
Name: gain, dtype: float64

In [7]:
bids

,quantity,price,user,buying,time,divisible
0,1,3,0,True,0,True
1,2,4,1,True,0,True
2,5,1,2,True,0,True
3,4,2,3,False,0,True
4,1,1,4,False,0,True
5,5,6,5,False,0,True


In [8]:
tmp = bids.reset_index().rename(columns={'index': 'bid'})

In [9]:
aux = {0: 1, 3:5}

In [10]:
tmp

,bid,quantity,price,user,buying,time,divisible
0,0,1,3,0,True,0,True
1,1,2,4,1,True,0,True
2,2,5,1,2,True,0,True
3,3,4,2,3,False,0,True
4,4,1,1,4,False,0,True
5,5,5,6,5,False,0,True


In [11]:
tmp.apply(lambda x: aux.get(x.user, x.price), axis=1)

0    1
1    4
2    1
3    5
4    1
5    6
dtype: int64

In [12]:
reservation_prices = {}
buyers = bids.loc[bids['buying']].index.values
sellers = bids.loc[~bids['buying']].index.values
#print(buyers, sellers)
index = [(i, j) for i in buyers for j in sellers]
#print(index)
for i, x in bids.iterrows():
    if i not in reservation_prices:
        reservation_prices[i] = x.price

In [13]:
coefficients = {}
for x in index:
    coefficients[x] = reservation_prices[x[0]] - reservation_prices[x[1]]

coefficients2 = {}
for x in index:
    coefficients2[x] = bids.iloc[x[0], 1] - bids.iloc[x[1], 1]


In [14]:
print(coefficients, coefficients2)

{(0, 3): 1, (0, 4): 2, (0, 5): -3, (1, 3): 2, (1, 4): 3, (1, 5): -2, (2, 3): -1, (2, 4): 0, (2, 5): -5} {(0, 3): 1, (0, 4): 2, (0, 5): -3, (1, 3): 2, (1, 4): 3, (1, 5): -2, (2, 3): -1, (2, 4): 0, (2, 5): -5}


In [15]:
for x in index:
    print(x, bids.iloc[x[0], 1]- bids.iloc[x[1], 1], reservation_prices[x[0]] - reservation_prices[x[1]])
reservation_prices

(0, 3) 1 1
(0, 4) 2 2
(0, 5) -3 -3
(1, 3) 2 2
(1, 4) 3 3
(1, 5) -2 -2
(2, 3) -1 -1
(2, 4) 0 0
(2, 5) -5 -5


{0: 3, 1: 4, 2: 1, 3: 2, 4: 1, 5: 6}

In [16]:
yy

NameError: name 'yy' is not defined

## Orignal supply and demand curves

In [ ]:
mar.plot()

## Supply and demand curves after market is splitted

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
plot_demand_curves(bids[bids.index.isin(extras['left'])], ax=ax[0])
ax[0].axhline(extras['price_right'], linestyle='--', c='k', label='Trading price determined by right players')
ax[0].legend()
ax[0].set_title('Left players')

plot_demand_curves(bids[bids.index.isin(extras['right'])], ax=ax[1])
ax[1].axhline(extras['price_left'], linestyle='--', c='k', label='Trading price determined by left players')
ax[1].legend()
_ = ax[1].set_title('Right players')

## Analysis of the Left Side

### Participants

* Buying: 1, 3, 4
* Selling: 7, 8, 9

### Trading price

* $6.3$

### Results

* The long side is the supply, all demand side buys as much as they want
* The demand side pays no fees, they are the short side
* Bid 7, results in bid 9 not trading a unit, so the fee is $ 1 \times (6.3 - 4) = 2.3$
* Bid 8, results in bid 9 not trading a 2 units so the fee is $2 \times (6.3 - 4) = 4.6$

## Analysis of the Right Side

### Participants

* Buying: 0, 2, 5
* Selling: 6, 10 (10 does not trade because bid price is greater than trading price)

### Trading price

* 4.65

### Results

* The long side is the demand, all supply side buys as much as they want
* The supply side pays no fees, they are the short side
* Bid 0, results in bid 2 not trading a unit, so the fee is $ 1 \times (6.5 - 4.65) = 1.85$

In [ ]:
reservation_prices = {}
buyers = bids.loc[bids['buying']].index.values
sellers = bids.loc[~bids['buying']].index.values
print(buyers, sellers)
index = [(i, j) for i in buyers for j in sellers]
print(index)
for i, x in bids.iterrows():
    print(x.price, bids.iloc[i, 1])
    if i not in reservation_prices:
        reservation_prices[i] = x.price

In [ ]:
reservation_prices

In [ ]:
for x in index:
    print(x, bids.iloc[x[0], 1] - bids.iloc[x[1], 1])
    print(x, reservation_prices[x[0]] - reservation_prices[x[1]])

In [ ]:
for i, b in bids.iterrows():
    print(i, b.price)

In [ ]:
transactions.get_df().values

In [ ]:
extras['fees']

In [ ]:
bids

In [ ]:
extra['left']

In [ ]:
extra['right']